<a href="https://colab.research.google.com/github/dnguyend/MiscCollection/blob/main/SphereSymplecticIntegrator/colab/SphereSymplectic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\cQ}{\mathcal{Q}}$
# Code for a symplectic integrator on the unit sphere $\cQ=S^{N-1}\in \R^N$
based on 
https://link.springer.com/content/pdf/10.1007/s10208-022-09571-x.pdf

* Proper implementation of Hamilton vector fields on an embedded manifold and the retraction on $TT^*\cQ$.

* We implement the lift of the retraction to $TT^*\cQ$ and show numerically it is a symplectomorphism (aka. canonical transformation) by showing the sympletic pairing is preserved.
* We implement an integrator based on the paper and show numerically it is also a symplectomorphism, preserving the pairing.


* The result can be viewed online or run in the colab environment. To run you need a gmail account. You can view the results without running.
* Code is in Python. We implement basic methods in a class Sphere, representing a sphere $\cQ = S^{N-1}$ with the retraction. Method randSymp generate a point on $T^*\cQ$, randTSymp a tangent vector in $TT^*\cQ$, $RTs0$ is $R^{T^*}$ described in the paper, with the adjustment accounting for the fact that we are dealing with embedded coordinate, for with $\theta =0$, $InvRTs0$ its inverse and $HamiltonVField$ translates an ambient vector field to a vector field on $T^*\cQ$.


In [1]:
import numpy as np
import numpy.linalg as la
from scipy.optimize import fsolve

class Sphere(object):
    def __init__(self, N):
        self.N = N

    def rand(self):
        Q = np.random.randn(self.N)
        return Q/la.norm(Q)

    def rand_ambient(self):
        return np.random.randn(self.N)

    def proj(self, Q, omg):
        return omg - Q*np.sum(Q*omg)

    def Dproj(self, Q, xi, omg):
        return -xi*np.sum(Q*omg) - Q*np.sum(xi*omg)

    def randvec(self, Q):
        """A random tangent vector at Q
        """
        return self.proj(Q, np.random.randn(self.N))

    def randp(self, Q):
        """A random cotangent vector at Q. In this case it is just a tangent vector
        """
        return self.proj(Q, np.random.randn(self.N))
    
    def rtr(self, Q, Dq):        
        return (Q + Dq) / np.sqrt(1 + np.sum(Dq*Dq))
    
    def inner2(self, pomg1, pomg2):
        return np.sum(pomg1[0]*pomg2[0]) + np.sum(pomg1[1]*pomg2[1])
                    
    def randSymp(self):
        """random element of T^*M
        """
        Q = self.rand()
        P = self.randp(Q)
        return Q, P

    def randTSymp(self, Q, P):
        """random element of T_{Q,P}T^*M
        """
        Dq = self.randvec(Q)
        tp = self.randp(Q)
        return Dq, tp - Q*np.sum(Dq*P)

    def RTs0(self, Q, P, Dq, Dp):
        """discretization on T*M with $\theta = 0$
        """
        Q0 = Q
        Q1 = self.rtr(Q, Dq)
        P0 = np.sum(Dq*P)*Dq + P - Q*np.sum(Dq*P) - Dp
        P1 = np.sqrt(1 + np.sum(Dq*Dq))*(P - Q*np.sum(Dq*P))
        return Q0, P0, Q1, P1

    def InvRTs0(self, Q0, P0, Q1, P1):
        """This one is used in the integrator
        """
        Qh = Q0
        Dq = Q1/np.sum(Q0*Q1) - Q0
        Ph = np.sum(Q0*Q1)*(P1 - Q0*np.sum(Q0*P1))
        Dpp = -np.sum(Q0*P1)*Q1 +\
            np.sum(Q0*Q1)*P1 - P0 \
            + Q0*np.sum(Q0*Q1)*np.sum(Q0*P1)
        return Qh, Ph, Dq, Dpp

    def HamiltonVField(self, Q, P, egradq, egradp):
        """Hamilton vector field. egradq and egradp form the ambient gradient
        of the Hamiltonian H.
        """
        Dq = self.proj(Q, egradp)
        Dp = - self.proj(Q, egradq) +  P*np.sum(Q*egradp) - Q*np.sum(P*egradp)
        return Dq, Dp


Do various sanity check. Confirming for example the randomly generate tangent/cotangent vectors satisfying the constrained of the bundles.

In [2]:
N = 5
Sp = Sphere(N)
Q = Sp.rand()
print(np.sum(Q*Q))
P = Sp.randp(Q)
V = Sp.randvec(Q)
print(np.sum(Q*P))
print(np.sum(Q*V))
omg = Sp.rand_ambient()
print(np.sum(Q*Sp.proj(Q, omg)))
Qr = Sp.rtr(Q, V)
np.sum(Qr*Qr)
Q, P = Sp.randSymp()
print(np.sum(Q*Q))
print(np.sum(Q*P))
Dq, Dp = Sp.randTSymp(Q, P)
print(np.sum(Q*Dq))
print(np.sum(Dq*P) + np.sum(Q*Dp))
Q0, P0, Q1, P1 = Sp.RTs0(Q, P, Dq, Dp)
print(np.sum(Q0*Q0))
print(Q0-Q)
print(np.sum(Q0*P0))
print(np.sum(Q1*Q1))
print(np.sum(Q1*P1))
Qh, Ph, Dqh, Dph = Sp.InvRTs0(Q0, P0, Q1, P1)
print(Qh-Q, Ph-P, Dqh-Dq, Dph-Dp)

1.0
8.326672684688674e-17
-1.1102230246251565e-16
-2.7755575615628914e-17
1.0
-2.7755575615628914e-16
1.6653345369377348e-16
0.0
1.0
[0. 0. 0. 0. 0.]
-1.0408340855860843e-16
1.0000000000000002
-7.494005416219807e-16
[0. 0. 0. 0. 0.] [-4.44089210e-16  2.22044605e-16  1.11022302e-16  4.64905892e-16
  2.22044605e-16] [ 6.66133815e-16 -1.11022302e-16 -5.55111512e-17 -1.11022302e-16
 -5.55111512e-17] [-7.77156117e-16 -4.44089210e-16 -6.66133815e-16 -1.11022302e-15
  4.44089210e-16]


# Show the map RTs0 is symplectic.
Generate random tangent vector to $TT^*\cQ$ at $(Q, P, Dq, Dp)$. Note the constraints on $Q, P, Dq, Dp$ are:
$$Q.Q = 1\\
Q.P =0\\
Q.Dq = 0\\
Dq.P + Q.Dp = 0.
$$
A tangent vector to $TT^*\cQ$ is a quaduple $(dQ, dP, dDq, dDp)$ satisfying the derived constraints of the above
$$Q.dQ =0\\
dQ.P + Q.dP = 0\\
dQ.Dq + Q.dDq =0\\
dDq.P + Dq.dP+ dQ.Dp + Q.dDp = 0.
$$
Note that $randvec(Q)$ generates a random tangent vector to the sphere $\cQ=S^{N-1}$ at $Q\in\cQ$. So we can generate random $dQ, dP, dDq$ and $dDp$ by
$$dQ = randvec(Q)\\
dP = randvec(Q) - (dQ.P)Q\\
dDq = randvec(Q) - (dQ.Dq)Q\\
dDp = randvec(Q) - (dDq.P + Dq.dP+ dQ.Dp)Q$$
The relations are verified as $Q.randvec(Q) = 0$ and $Q.Q = 1$
* We generate two random tangent vectors
$(dQ0, dP0, dDq0, dDp0)$, $(dQ1, dP1, dDq1, dDp1)$ to $TT^*\cQ$ at $(Q, P, Dq, Dp)$, compute their symplectic pairing $prodTTs$. Then we compute 
$diff0 = (RTs0(Q+\delta dQ0, P+\delta dP0, Dq+\delta dDq0, Dp+dDp0)-
RTs0(Q, P, Dq, Dp))/dlt$
$diff1 = (RTs0(Q+\delta dQ1, P+\delta dP1, Dq+\delta dDq1, Dp+dDp1)-
RTs0(Q, P, Dq, Dp))/dlt$
which are two tangent vectors of $T^*\cQ\times T^*\cQ$. The pairing between $diff0, diff1$ and the original pairing $prodTTs$ are the same.


In [3]:
# np.random.seed(0)
Q, P = Sp.randSymp()
Dq, Dp = Sp.randTSymp(Q, P)

def randvecTTs(Sp, Q, P, Dq, Dp):
  dQ = Sp.randvec(Q)
  dP = Sp.randvec(Q) - np.sum(dQ*P)*Q
  dDq = Sp.randvec(Q) - np.sum(dQ*Dq)*Q
  dDp = Sp.randvec(Q) - (np.sum(dDq*P) + np.sum(Dq*dP) + np.sum(dQ*Dp))*Q
  return dQ, dP, dDq, dDp
dQ0, dP0, dDq0, dDp0 = randvecTTs(Sp, Q, P, Dq, Dp)
dQ1, dP1, dDq1, dDp1 = randvecTTs(Sp, Q, P, Dq, Dp)

dlt = 1e-7
diff0 = (np.concatenate(Sp.RTs0(Q+dlt*dQ0, P+dlt*dP0, Dq + dlt*dDq0, Dp + dlt*dDp0)) -
      np.concatenate(Sp.RTs0(Q, P, Dq, Dp)))/dlt
diff1 = (np.concatenate(Sp.RTs0(Q+dlt*dQ1, P+dlt*dP1, Dq + dlt*dDq1, Dp + dlt*dDp1)) -
      np.concatenate(Sp.RTs0(Q, P, Dq, Dp)))/dlt

prodTTs = np.sum(dQ1*dDp0) + np.sum(dDq1*dP0) -\
      np.sum(dQ0*dDp1) - np.sum(dDq0*dP1)

prodTstimesTs = -np.sum(diff0[N:2*N]*diff1[:N]) + np.sum(diff0[:N]*diff1[N:2*N]) +\
      np.sum(diff0[3*N:]*diff1[2*N:3*N]) - np.sum(diff0[2*N:3*N]*diff1[3*N:])

display("pairing in TT^*Q =%f, in T^*Q times T^*Q=%f" % (prodTTs, prodTstimesTs))


'pairing in TT^*Q =4.510759, in T^*Q times T^*Q=4.510752'

# The Symplectic integrator.
The function HamiltonVField translates the Euclidean gradient $H_q, H_p$ to vector fields $X^H_q, X^H_p$ on $TT^*Q$.

With $\hat{Q} = Q_0$, and $\Delta_q = h X^H_q, \Delta_p = h X^H_p$, 

we solve for $\hat{P}\in T^*_{Q_0}Q$ satisfying the equation
$$P_0 = (\Delta_q.\hat{P})\Delta_q + \hat{P} - (\Delta_q. \hat{P})\hat{Q} -\Delta_p 
$$
Then $Q_1, P_1$ are computed from the formula for $R^{T^*}$.

In [4]:
from scipy.optimize import fsolve

def calc_p(Sp, Q0, P0, H_q, H_p, h):
    vv0 = np.concatenate([P0, np.zeros(1)])

    def F(vv):
      hatp = vv[:-1]
      lbd = vv[-1]
      Dq, Dp = Sp.HamiltonVField(
          Q0, hatp, H_q(Q0, hatp), H_p(Q0, hatp))
      # _, ret2, _, _ = Sp.RTs(Q0, hatp, h*DDq, h*DDp, 0.)
      ret2 = h*h*np.sum(Dq*hatp)*Dq + hatp - h*Q0*np.sum(Dq*hatp) - h*Dp
      # print(ret2)
      return np.concatenate(
          [ret2 - P0 - Q0*lbd, np.array([np.sum(Q0*vv[:-1])])])

    ret = fsolve(F, vv0)
    return ret[:-1]

def calc_p0(Sp, qq, pp, H_q, H_p, h):
    vv0 = np.concatenate([pp, np.zeros(1)])

    def F(vv):
      hatp = vv[:-1]
      lbd = vv[-1]

      DDq, DDp = Sp.HamiltonVField(
          qq, hatp, H_q(qq, hatp), H_p(qq, hatp))
      _, ret2, _, _ = Sp.RTs0(qq, hatp, h*DDq, h*DDp)
      # print(ret2)
      return np.concatenate(
          [ret2 - pp - qq*lbd, np.array([np.sum(qq*vv[:-1])])])

    ret = fsolve(F, vv0)
    return ret[:-1]


def integrator(Sp, Qk, Pk, H_q, H_p, h):
    hatp = calc_p(Sp, Qk, Pk, H_q, H_p, h)
    Hpv = H_p(Qk, hatp)
    Hqv = H_q(Qk, hatp)

    Dq, Dp = Sp.HamiltonVField(Qk, hatp, Hqv, Hpv)
    Qold, Pold, Qk1, Pk1 = Sp.RTs0(Qk, hatp, h*Dq, h*Dp)
    # Qk1 = Sp.rtr(Qk, h*Dq)
    # Pk1 = np.sqrt(1 + h*h*np.sum(Dq*Dq))*(hatp - h*Qk*np.sum(Dq*hatp))
    return Qk1, Pk1

# Test the integrator is symplectic with several Hamiltonian functions.
We show the integrator is symplectic, preserving the symplectic pairing

In [5]:
A = np.random.randn(N, N)
A = A@A.T

def U(Q):
  return 0.5*np.sum(Q*(A@Q))

def Uq(Q):
  return A@Q

def H1(Q, P):
  return 0.5*np.sum(P*P) + U(Q)

def H_q1(Q, P):
  return Uq(Q)

def H_p1(Q, P):
  return P

Q0, P0 = Sp.randSymp()
Dx1, Dp1 = Sp.randTSymp(Q0, P0)
Dx2, Dp2 = Sp.randTSymp(Q0, P0)

dlt = 1e-7
h = 1e-2
Qnew0, Pnew0 = integrator(Sp, Q0, P0, H_q1, H_p1, h)
Qnew1, Pnew1 = integrator(Sp, Q0+dlt*Dx1, P0+dlt*Dp1, H_q1, H_p1, h)
Qnew2, Pnew2 = integrator(Sp, Q0+dlt*Dx2, P0+dlt*Dp2, H_q1, H_p1, h)

diff1 = (np.concatenate([Qnew1, Pnew1]) - np.concatenate([Qnew0, Pnew0]))/dlt
diff2 = (np.concatenate([Qnew2, Pnew2]) - np.concatenate([Qnew0, Pnew0]))/dlt
Pair0 = np.sum(Dx1*Dp2) - np.sum(Dx2*Dp1)
PairNew = np.sum(diff1[:N]*diff2[N:]) - np.sum(diff2[:N]*diff1[N:])
print("Original pairing = %f, transformed pairing = %f" % (Pair0, PairNew))


Original pairing = -2.302279, transformed pairing = -2.302279


Another Hamitonian function with more complicated kinetic

In [6]:
# np.random.seed(0)
A = np.random.randn(N, N)
A = A@A.T

def U(Q):
  return 0.5*np.sum(Q*(A@Q))

def Uq(Q):
  return A@Q

def H2(Q, P):
  return 0.5*np.sum(P*P*P*P) + 0.5*np.sum(P*Q*Q*P) + U(Q)

def H_q2(Q, P):
  return Uq(Q) + P*P*Q

def H_p2(Q, P):
  return 2*P*P*P + P*Q*Q

Q0, P0 = Sp.randSymp()
Dx1, Dp1 = Sp.randTSymp(Q0, P0)
Dx2, Dp2 = Sp.randTSymp(Q0, P0)

dlt = 1e-7
h = 1e-2
Qnew0, Pnew0 = integrator(Sp, Q0, P0, H_q2, H_p2, h)
Qnew1, Pnew1 = integrator(Sp, Q0+dlt*Dx1, P0+dlt*Dp1, H_q2, H_p2, h)
Qnew2, Pnew2 = integrator(Sp, Q0+dlt*Dx2, P0+dlt*Dp2, H_q2, H_p2, h)

diff1 = (np.concatenate([Qnew1, Pnew1]) - np.concatenate([Qnew0, Pnew0]))/dlt
diff2 = (np.concatenate([Qnew2, Pnew2]) - np.concatenate([Qnew0, Pnew0]))/dlt
Pair0 = np.sum(Dx1*Dp2) - np.sum(Dx2*Dp1)
PairNew = np.sum(diff1[:N]*diff2[N:]) - np.sum(diff2[:N]*diff1[N:])
print("Original pairing = %f, transformed pairing = %f" % (Pair0, PairNew))


Original pairing = -0.031209, transformed pairing = -0.031209
